In [8]:
import pandas as pd, geopandas as gpd
from pathlib import Path

acd_path = "/Users/shujaali/Downloads/github/data/UcdpPrioConflict_v25_1.csv"  # <- your path

# ACD → country-year intensity
acd = pd.read_csv(acd_path, low_memory=False)
acd = acd.loc[acd["year"] >= 1946, ["location","year","intensity_level"]].rename(
    columns={"intensity_level":"intensity"}
)
cy = acd.groupby(["location","year"], as_index=False)["intensity"].max()

# Natural Earth polygons
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world = world[world["name"] != "Antarctica"][["name","iso_a3","geometry"]].copy()

/var/folders/sk/p0bpqc995x910krmdwnwxsy00000gn/T/ipykernel_50705/3716437653.py:14: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))


In [9]:
# Light name harmonization so we can attach ISO codes
name_fix = {
    "Congo, Dem. Rep.": "Democratic Republic of the Congo",
    "Congo, Rep.": "Republic of the Congo",
    "Côte d'Ivoire": "Ivory Coast",
    "Eswatini": "Swaziland",
    "Czechia": "Czech Republic",
    "North Macedonia": "Macedonia",
    "United States of America": "United States",
    "Russian Federation": "Russia",
    "Syrian Arab Republic": "Syria",
    "Iran": "Iran (Islamic Republic of)",
    "Viet Nam": "Vietnam",
    "Lao PDR": "Laos",
    "Tanzania": "United Republic of Tanzania",
    "Bahamas": "The Bahamas",
    "Cabo Verde": "Cape Verde",
    "Bosnia and Herz.": "Bosnia and Herzegovina",
    "Myanmar": "Myanmar (Burma)",
    "The Gambia": "Gambia",
    "Micronesia": "Micronesia (Federated States of)",
    "Bolivia": "Bolivia (Plurinational State of)",
}
world["join_loc"] = world["name"].replace(name_fix)
cy = cy.assign(join_loc=cy["location"])

# Complete country×year panel (fill missing intensity = 0)
years = list(range(1946, int(cy["year"].max())+1))
countries = world["join_loc"].tolist()
panel = pd.MultiIndex.from_product([countries, years], names=["join_loc","year"]).to_frame(index=False)

cy_full = (
    panel.merge(cy[["join_loc","year","intensity"]], on=["join_loc","year"], how="left")
         .assign(intensity=lambda d: d["intensity"].fillna(0).astype(int))
)

# Attach geometry & ISO3
world_year = world[["join_loc","name","iso_a3","geometry"]].merge(cy_full, on="join_loc", how="right")
world_year = gpd.GeoDataFrame(world_year, geometry="geometry", crs=world.crs)
world_year = world_year.dropna(subset=["geometry"]).to_crs(4326)

In [11]:
import pycountry
from unidecode import unidecode

def iso3_to_en(iso):
    if pd.isna(iso): return None
    iso = str(iso)
    overrides = {"XKX":"Kosovo","KOS":"Kosovo","PSE":"Palestine","SSD":"South Sudan",
                 "SDS":"South Sudan","TWN":"Taiwan","ESH":"Western Sahara","-99":None}
    if iso in overrides: return overrides[iso]
    rec = pycountry.countries.get(alpha_3=iso)
    return rec.name if rec else None

# 1) ISO3 → English, 2) fallback to NE name, 3) fallback to join_loc; then transliterate
label_en = world_year["iso_a3"].map(iso3_to_en)
ne_name_by_iso = dict(zip(world["iso_a3"], world["name"]))
label_en = label_en.fillna(world_year["iso_a3"].map(ne_name_by_iso))
label_en = label_en.fillna(world_year["join_loc"])
world_year["label_ascii"] = label_en.apply(lambda s: unidecode(str(s)) if pd.notna(s) else s)

In [ ]:
import json
import geopandas as gpd

def year_geojson(df, year):
    g = df[df["year"] == year][["label_ascii","intensity","geometry"]].copy()
    feats=[]
    for _, r in g.iterrows():
        geom = r.geometry
        if geom is None or geom.is_empty: 
            continue
        geom_js = json.loads(gpd.GeoSeries([geom]).to_json())["features"][0]["geometry"]
        feats.append({
            "type":"Feature",
            "geometry": geom_js,
            "properties": {
                "name": r["label_ascii"],
                "intensity": int(r["intensity"]),
            }
        })
    return {"type":"FeatureCollection","features":feats}

years_sorted = sorted(world_year["year"].unique().tolist())
gj_cache = {y: year_geojson(world_year, y) for y in years_sorted}
from ipyleaflet import Map, GeoJSON, LayersControl, basemaps
from ipywidgets import IntSlider, VBox, HTML

COLOR = {0:"#d9d9d9", 1:"#fdae6b", 2:"#de2d26"}  # none / minor / war

def style_callback(feature):
    val = int(feature["properties"]["intensity"])
    return {"fillColor": COLOR.get(val, "#d9d9d9"),
            "color":"white", "weight":0.2, "fillOpacity":0.75}

# Latin-only labels basemap
m = Map(center=(20,0), zoom=2, basemap=basemaps.CartoDB.Positron, scroll_wheel_zoom=True)

start_year = 1946
layer = GeoJSON(
    data=gj_cache[start_year],
    style={"color":"white","weight":0.2,"fillOpacity":0.75},  # dict
    style_callback=style_callback,                             # function for per-feature color
    hover_style={"weight":1,"fillOpacity":0.85}
)
m.add_layer(layer)
m.add_control(LayersControl(position="topright"))

slider = IntSlider(description="Year", min=years_sorted[0], max=years_sorted[-1],
                   step=1, value=start_year, continuous_update=False)
label = HTML(f"<b>UCDP Conflict Intensity — {start_year}</b>")

def on_change(ch):
    if ch["name"] == "value":
        y = int(ch["new"])
        layer.data = gj_cache[y]
        label.value = f"<b>UCDP Conflict Intensity — {y}</b>"

slider.observe(on_change, names="value")

# Display the map with slider
VBox([label, slider, m])